In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
!pip install transformers==4.47.1 accelerate==0.34.2 bitsandbytes==0.45.0 trl==0.13.0 datasets==3.2.0 peft==0.14.0 tokenizers==0.21.0 huggingface_hub==0.26.0 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 68.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 28.4 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, BitsAndBytesConfig
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from peft import PeftModel,get_peft_model,LoraConfig, TaskType
from trl import SFTTrainer, SFTConfig

In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HuggigFace") # Fetching the Hugging Face token from the Kaggle Secret keys add on
login(token = hf_token) # Logging into Hugging 

### setup model and bnb config

In [5]:
base_model = "meta-llama/Llama-3.2-3B-Instruct"

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable loading the model with 4-bit precision for reduced memory usage
    bnb_4bit_quant_type='nf4',  # Use NormalFloat4 (nf4), a quantization format for higher accuracy
    bnb_4bit_compute_dtype=torch.float16,  # Use float16 for computation to balance speed and precision
    bnb_4bit_use_double_quant=True  # Enable double quantization for better numerical stability
)

# Load the pre-trained model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    base_model,  # Name of the base model defined earlier
    device_map="auto",  # Automatically map model layers to available devices (e.g., GPU/CPU)
    quantization_config=bnb_config,  # Apply the defined 4-bit quantization configuration
)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [7]:
# add tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

### Load the dataset

In [21]:
db = "NebulaByte/E-Commerce_Customer_Support_Conversations"

In [22]:
# find the customer support dataset and load into the cell
dataset = load_dataset(db, split="train")

In [23]:
dataset

Dataset({
    features: ['issue_area', 'issue_category', 'issue_sub_category', 'issue_category_sub_category', 'customer_sentiment', 'product_category', 'product_sub_category', 'issue_complexity', 'agent_experience_level', 'agent_experience_level_desc', 'conversation'],
    num_rows: 1000
})

In [24]:
dataset = dataset.remove_columns(['issue_area','issue_sub_category', 'issue_category_sub_category',
                                  'customer_sentiment', 'product_category', 'product_sub_category', 
                                  'issue_complexity', 'agent_experience_level'])

In [25]:
dataset.train_test_split(test_size=0.20)

DatasetDict({
    train: Dataset({
        features: ['issue_category', 'agent_experience_level_desc', 'conversation'],
        num_rows: 800
    })
    test: Dataset({
        features: ['issue_category', 'agent_experience_level_desc', 'conversation'],
        num_rows: 200
    })
})

### Tokenize the dataset

In [26]:
# set the instructions 
instruction = """You are helpful and efficient e-commerce customer support assistant bot designed to assist users by providing answers to frequently asked questions (FAQs) related to our products and services. Your responses should be concise, clear, and friendly, ensuring the user feels heard and supported. If the user’s question is outside the scope of the FAQ, gently direct them to contact customer support.

Always prioritize accuracy and clarity in your answers.
If the user asks a complex question, break it down into smaller, manageable parts and answer step-by-step.
Provide useful links or references to detailed documentation when appropriate.
Use a friendly and professional tone, ensuring the response is easy to understand.
If the FAQ does not cover the question, offer an apology and suggest contacting customer support.
"""

def template(row):

    row_json = [{"role":"system", "content":instruction},
                {"role":"category", "content":row["issue_category"]},
                {"role":"user", "content":row["agent_experience_level_desc"]},
                {"role":"assistant", "content": row["conversation"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

In [27]:
dataset = dataset.map(template,num_proc= 4)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [36]:
dataset = dataset.train_test_split(test_size=0.20)

In [39]:
dataset["train"]["text"][0]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 02 Jan 2025\n\nYou are helpful and efficient e-commerce customer support assistant bot designed to assist users by providing answers to frequently asked questions (FAQs) related to our products and services. Your responses should be concise, clear, and friendly, ensuring the user feels heard and supported. If the user’s question is outside the scope of the FAQ, gently direct them to contact customer support.\n\nAlways prioritize accuracy and clarity in your answers.\nIf the user asks a complex question, break it down into smaller, manageable parts and answer step-by-step.\nProvide useful links or references to detailed documentation when appropriate.\nUse a friendly and professional tone, ensuring the response is easy to understand.\nIf the FAQ does not cover the question, offer an apology and suggest contacting customer support.<|eot_id|><|start_header_id|>category<|end_h

### set the lora config and train the model

In [41]:
lora_config = LoraConfig(r=4, 
                         lora_alpha=8, 
                         lora_dropout=0.2,
                        task_type="CAUSAL_LM")

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 1,146,880 || all params: 3,213,896,704 || trainable%: 0.0357


In [43]:
# setting up training arguments for the model training process
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=5,
    learning_rate=2e-4,
    fp16=True,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
)

<ipython-input-43-8c8846e77e81>:13: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [44]:
model.train()
trainer.train()

Step,Training Loss
500,0.595800


TrainOutput(global_step=800, training_loss=0.5496325302124023, metrics={'train_runtime': 615.1812, 'train_samples_per_second': 1.3, 'train_steps_per_second': 1.3, 'total_flos': 9422225526804480.0, 'train_loss': 0.5496325302124023, 'epoch': 1.0})

### Testing the model

In [49]:
def generate(category, input_prompt):
    messages = [
        {"role":"system", "content":instruction},
        {"role":"category","content":category},
        {"role":"user","content":input_prompt}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=2048, num_return_sequences=1)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

In [50]:
res = generate("Lost or Missing Warranty Card", "What should I do to get the lost warranty card?")
print(res)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Cutting Knowledge Date: December 2023
Today Date: 02 Jan 2025

You are helpful and efficient e-commerce customer support assistant bot designed to assist users by providing answers to frequently asked questions (FAQs) related to our products and services. Your responses should be concise, clear, and friendly, ensuring the user feels heard and supported. If the user’s question is outside the scope of the FAQ, gently direct them to contact customer support.

Always prioritize accuracy and clarity in your answers.
If the user asks a complex question, break it down into smaller, manageable parts and answer step-by-step.
Provide useful links or references to detailed documentation when appropriate.
Use a friendly and professional tone, ensuring the response is easy to understand.
If the FAQ does not cover the question, offer an apology and suggest contacting customer support.category

Lost or Missing Warranty Carduser

What should I do to get the lost warranty card?assistant

I'm so

In [51]:
model.save_pretrained("/content/ecom_customer_support_chatbot_llama3.2-3B")

In [53]:
# push model to hf hub
model.push_to_hub("ecom-customer-support-llama3-2-3B-fine-tuned")

adapter_model.safetensors:   0%|          | 0.00/4.60M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/avikumart/ecom-customer-support-llama3-2-3B-fine-tuned/commit/74e0aa5b5696659777ff2a965a4e6b8a1d48645f', commit_message='Upload model', commit_description='', oid='74e0aa5b5696659777ff2a965a4e6b8a1d48645f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/avikumart/ecom-customer-support-llama3-2-3B-fine-tuned', endpoint='https://huggingface.co', repo_type='model', repo_id='avikumart/ecom-customer-support-llama3-2-3B-fine-tuned'), pr_revision=None, pr_num=None)